## PM2.5 Concentration Prediction with Linear Regression & XGBoost

In [1]:
import numpy as np
import pandas as pd

In [2]:
import warnings
warnings.simplefilter("ignore")

In [3]:
data = pd.read_excel("data.xlsx")

### Data Preprocessing

In [4]:
data = data[4914:]

In [5]:
# convert specific value to "F"

labels = ["*", "#", "x", "A"]

for time in range(0, 24):
    for label in labels:
        for index in data[data[time]==label].index:
            data.loc[index, time] = "F"

In [6]:
data.shape

(1656, 27)

In [7]:
data[data["測項"]=="AMB_TEMP"].head()

,測站,日期,測項,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
4914,新竹,2021-10-01,AMB_TEMP,28.3,28.3,27.8,27.8,27.6,27.6,27.7,...,31.6,31.4,30.9,30.5,30.2,29.8,29.4,29.1,28.7,28.2
4932,新竹,2021-10-02,AMB_TEMP,27.8,27.4,27.1,27,26.9,26.7,26.5,...,32.2,31.9,31.1,30.7,30,29.3,28.8,28.4,28.2,28
4950,新竹,2021-10-03,AMB_TEMP,27.8,27.7,27.6,27.2,27.5,27.1,26.9,...,33.9,34.1,33.4,32.5,31.8,31,30,30.2,29.6,28.4
4968,新竹,2021-10-04,AMB_TEMP,27.5,27.9,27.7,27,26.8,26.4,26.3,...,32.3,32,31.4,30.5,30.3,30.1,30,29.5,29.1,29
4986,新竹,2021-10-05,AMB_TEMP,28.8,28.8,28.7,28.7,28.7,28.7,28.7,...,35.1,34.2,33.2,32.5,31.2,30.6,30,29.5,29.4,29.7


In [8]:
time_list = []
column_list = [] # for index of df

for date in data["日期"].unique():
    time_list.append(str(date).split("T")[0].replace("2021-", ""))
    
for t in time_list:
    for d in data.columns[3:]:
        column_list.append(t+"-"+str(d))

In [9]:
# 18 pollutions
features = list(data["測項"].unique())

# create the dataframe first
df = pd.DataFrame()

for feature in features:
    data_list = []
    columns   = data[data["測項"]==feature].transpose()[3:].columns

    for column in columns:
        for d in data[data["測項"]==feature].transpose()[3:][column]:
            data_list.append(d)

    df[str(feature)] = data_list

In [10]:
df.head()

,AMB_TEMP,CH4,CO,NMHC,NO,NO2,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,THC,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR
0,28.3,2.04,0.34,0.17,0.9,18.8,19.8,16,28,28,0,71,1.8,2.21,62,33,0.8,0.7
1,28.3,2.02,0.3,0.13,0.2,11.9,12.2,21.5,24,22,0,66,1,2.15,121,183,1.2,0.6
2,27.8,2.12,0.3,0.12,0.5,15.1,15.6,16.9,29,26,0,76,1,2.24,164,160,1.1,0.6
3,27.8,2.18,0.29,0.14,0.4,12.8,13.2,16.4,32,24,0,79,1.3,2.32,156,151,0.6,0.4
4,27.6,2.19,0.3,0.17,0.2,14.9,15.1,12.6,31,28,0,81,1.5,2.36,110,90,0.9,0.5


In [11]:
print(features)

['AMB_TEMP', 'CH4', 'CO', 'NMHC', 'NO', 'NO2', 'NOx', 'O3', 'PM10', 'PM2.5', 'RAINFALL', 'RH', 'SO2', 'THC', 'WD_HR', 'WIND_DIREC', 'WIND_SPEED', 'WS_HR']


In [12]:
import statistics

In [13]:
for feature in features:
    for index in df[df[feature]=="F"].index:
        if df.loc[index-1, feature] != "F" and df.loc[index+1, feature] != "F":
            # 前後一小時的平均值
            df.loc[index, feature] = (df.loc[index-1, feature]+df.loc[index+1, feature])/2
            # 若前一小時=0
        elif df.loc[index-2, feature] != "F" and df.loc[index+1, feature] != "F":
            df.loc[index, feature] = (df.loc[index-2, feature]+df.loc[index+1, feature])/2
            # 若前二小時=0
        elif df.loc[index-3, feature] != "F" and df.loc[index+1, feature] != "F":
            df.loc[index, feature] = (df.loc[index-3, feature]+df.loc[index+1, feature])/2
        else:  
            num_list = []
            
            for d in df.loc[:, feature]:
                if type(d) == float:
                    num_list.append(d)
            mean = statistics.mean(num_list)
            # 以平均填補
            df.loc[index, feature] = mean

In [14]:
df.index=column_list

### Training and Test Sets: Splitting Data

In [15]:
training_data = df.loc["10-01-0":"11-30-23", :]
testing_data  = df.loc["12-01-0":, :]

In [16]:
training_data = training_data.transpose()
testing_data  =  testing_data.transpose()

### Predict the next 1 hours

In [17]:
training_data.columns = [x for x in range(0, 1464)]
testing_data.columns  = [x for x in range(0,  744)]

In [18]:
training_data.loc["PM2.5", 0:5]

0    28
1    22
2    26
3    24
4    28
5    20
Name: PM2.5, dtype: object

In [19]:
x_train_1 = []
y_train_1 = []

for i in range(0, 1458):
    x_train_1.append(training_data.loc["PM2.5", i:i+5])
    y_train_1.append(training_data.loc["PM2.5",   i+6])

In [20]:
print(len(x_train_1))
print(len(y_train_1))

1458
1458


In [21]:
x_test_1 = []
y_test_1 = []

for i in range(0, 738):
    x_test_1.append(testing_data.loc["PM2.5", i:i+5])
    y_test_1.append(testing_data.loc["PM2.5",   i+6])

In [22]:
print(len(x_test_1))
print(len(y_test_1))

738
738


In [23]:
x_train_6 = []
y_train_6 = []

for i in range(0, 1453):
    x_train_6.append(training_data.loc["PM2.5", i:i+ 5])
    y_train_6.append(training_data.loc["PM2.5",   i+11])

In [24]:
print(len(x_train_6))
print(len(y_train_6))

1453
1453


In [25]:
x_test_6 = []
y_test_6 = []

for i in range(0, 733):
    x_test_6.append(testing_data.loc["PM2.5", i:i+ 5])
    y_test_6.append(testing_data.loc["PM2.5",   i+11])

In [26]:
print(len(x_test_6))
print(len(y_test_6))

733
733


In [27]:
x_train_1_all_features = []

for i in range(0, 1458):
    x_train_1_all_features.append(training_data.loc[:, i:i+5].transpose())

In [28]:
x_train_6_all_features = []

for i in range(0, 1453):
    x_train_6_all_features.append(training_data.loc[:, i:i+5].transpose())

In [29]:
print(x_train_1_all_features[0].shape)
print(x_train_6_all_features[0].shape)

(6, 18)
(6, 18)


In [30]:
x_test_1_all_features = []

for i in range(0, 738):
    x_test_1_all_features.append(testing_data.loc[:, i:i+5].transpose())

In [31]:
x_test_1_all_features[0]

,AMB_TEMP,CH4,CO,NMHC,NO,NO2,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,THC,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR
0,17.8,2.04,0.3,0.05,0.8,6.2,7,33.2,47,22,0,49,1.9,2.09,57,52,4.5,4.1
1,17.2,2.04,0.29,0.06,0.4,6.6,7,31.2,48,26,0,48,1.1,2.1,57,58,5.9,4.3
2,16.7,2.04,0.28,0.04,0.4,6,6.4,30.8,47,24,0,47,1.1,2.08,58,57,4.8,4.1
3,16.3,2.04,0.27,0.03,0.3,5.6,5.9,32.8,45,23,0,46,1.2,2.07,56,51,4.6,4.2
4,15.9,2.03,0.26,0.05,0.3,5.7,6.1,33,42,17,0,44,1.5,2.08,49,40,4.5,3.8
5,15.5,2.03,0.27,0.05,0.3,6.6,7,30.5,41,22,0,47,1.6,2.08,56,59,3.9,3.7


In [32]:
x_test_6_all_features = []

for i in range(0, 733):
    x_test_6_all_features.append(testing_data.loc[:, i:i+5].transpose())

In [33]:
print(x_test_1_all_features[0].shape)
print(x_test_6_all_features[0].shape)

(6, 18)
(6, 18)


### Use Linear Regression to predict the next 1 hours

In [34]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

# create an instance of Linear Regression
lin_reg = LinearRegression()

# use X and Y to fit the Regression Model
lin_reg.fit(x_train_1, y_train_1)
lin_reg.score(x_train_1, y_train_1)

0.6934301453772909

In [35]:
y_pred = lin_reg.predict(x_train_1)

# MSE & MAE
mse = mean_squared_error(y_train_1, y_pred)
mae = mean_absolute_error(y_train_1, y_pred)

print(f"MSE: {round(mse, 4)}")
print(f"MAE: {round(mae, 4)}")

MSE: 13.0626
MAE: 2.5907


In [36]:
y_pred = lin_reg.predict(x_test_1)

# MSE & MAE
mse = mean_squared_error(y_test_1, y_pred)
mae = mean_absolute_error(y_test_1, y_pred)

print(f"MSE: {round(mse, 4)}")
print(f"MAE: {round(mae, 4)}")

MSE: 14.1514
MAE: 2.6772


### Use XGBoost to predict the next 1 hours

In [37]:
# !pip install xgboost

In [38]:
import xgboost as xgb

# reg:linear >> reg:squarederror
xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

xgb_model.fit(x_train_1, y_train_1)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, ...)

In [39]:
y_pred = xgb_model.predict(x_train_1)

# MSE & MAE
mse = mean_squared_error(y_train_1, y_pred)
mae = mean_absolute_error(y_train_1, y_pred)

print(f"MSE: {round(mse, 4)}")
print(f"MAE: {round(mae, 4)}")

MSE: 1.0838
MAE: 0.6838


In [40]:
y_pred = xgb_model.predict(x_test_1)

# MSE & MAE
mse = mean_squared_error(y_test_1, y_pred)
mae = mean_absolute_error(y_test_1, y_pred)

print(f"MSE: {round(mse, 4)}")
print(f"MAE: {round(mae, 4)}")

MSE: 17.0863
MAE: 3.0722


### Use Linear Regression to predict the next 6 hours

In [41]:
lin_reg = LinearRegression()

lin_reg.fit(x_train_6, y_train_6)
lin_reg.score(x_train_6, y_train_6)

0.2960413032430438

In [42]:
y_pred = lin_reg.predict(x_train_6)

# MSE & MAE
mse = mean_squared_error(y_train_6, y_pred)
mae = mean_absolute_error(y_train_6, y_pred)

print(f"MSE: {round(mse, 4)}")
print(f"MAE: {round(mae, 4)}")

MSE: 29.1138
MAE: 3.8815


In [43]:
y_pred = lin_reg.predict(x_test_6)

# MSE & MAE
mse = mean_squared_error(y_test_6, y_pred)
mae = mean_absolute_error(y_test_6, y_pred)

print(f"MSE: {round(mse, 4)}")
print(f"MAE: {round(mae, 4)}")

MSE: 40.6797
MAE: 4.3067


### Use XGBoost to predict the next 6 hours

In [44]:
xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

xgb_model.fit(x_train_6, y_train_6)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, ...)

In [45]:
y_pred = xgb_model.predict(x_train_6)

# MSE & MAE
mse = mean_squared_error(y_train_6, y_pred)
mae = mean_absolute_error(y_train_6, y_pred)

print(f"MSE: {round(mse, 4)}")
print(f"MAE: {round(mae, 4)}")

MSE: 2.3765
MAE: 1.007


In [46]:
y_pred = xgb_model.predict(x_test_6)

# MSE & MAE
mse = mean_squared_error(y_test_6, y_pred)
mae = mean_absolute_error(y_test_6, y_pred)

print(f"MSE: {round(mse, 4)}")
print(f"MAE: {round(mae, 4)}")

MSE: 49.9791
MAE: 4.9842


### Modeling by Linear Regression and XGBoost (x=all_features)

In [47]:
# convert the dimension from 3 to 2
s1 = np.array(x_train_1_all_features).shape[0]
s2 = np.array(x_train_1_all_features).shape[1]
s3 = np.array(x_train_1_all_features).shape[2]

x_train_1_all_features_2D = np.array(x_train_1_all_features).reshape(s1, s2*s3)

len(x_train_1_all_features_2D[0])

108

In [48]:
s1 = np.array(x_test_1_all_features).shape[0]
s2 = np.array(x_test_1_all_features).shape[1]
s3 = np.array(x_test_1_all_features).shape[2]

x_test_1_all_features_2D = np.array(x_test_1_all_features).reshape(s1, s2*s3)

len(x_test_1_all_features_2D[0])

108

In [49]:
def dimension_conversion(dataset):
    s1 = np.array(dataset).shape[0]
    s2 = np.array(dataset).shape[1]
    s3 = np.array(dataset).shape[2]

    dataset_2D = np.array(dataset).reshape(s1, s2*s3)
    return dataset_2D

In [50]:
x_train_6_all_features_2D = dimension_conversion(x_train_6_all_features)
x_test_6_all_features_2D = dimension_conversion(x_test_6_all_features)

### Use Linear Regression to predict the next 1 hours

In [51]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

# create an instance of Linear Regression
lin_reg = LinearRegression()

# use X and Y to fit the Regression Model
lin_reg.fit(x_train_1_all_features_2D, y_train_1)
lin_reg.score(x_train_1_all_features_2D, y_train_1)

0.7507147219390069

In [52]:
y_pred = lin_reg.predict(x_train_1_all_features_2D)

# MSE & MAE
mse = mean_squared_error(y_train_1, y_pred)
mae = mean_absolute_error(y_train_1, y_pred)

print(f"MSE: {round(mse, 4)}")
print(f"MAE: {round(mae, 4)}")

MSE: 10.6218
MAE: 2.3878


In [53]:
y_pred = lin_reg.predict(x_test_1_all_features_2D)

# MSE & MAE
mse = mean_squared_error(y_test_1, y_pred)
mae = mean_absolute_error(y_test_1, y_pred)

print(f"MSE: {round(mse, 4)}")
print(f"MAE: {round(mae, 4)}")

MSE: 13.8619
MAE: 2.6476


### Use XGBoost to predict the next 1 hours

In [54]:
xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

xgb_model.fit(x_train_1_all_features_2D, y_train_1)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, ...)

In [55]:
y_pred = xgb_model.predict(x_train_1_all_features_2D)

# MSE & MAE
mse = mean_squared_error(y_train_1, y_pred)
mae = mean_absolute_error(y_train_1, y_pred)

print(f"MSE: {round(mse, 4)}")
print(f"MAE: {round(mae, 4)}")

MSE: 0.0069
MAE: 0.058


In [56]:
y_pred = xgb_model.predict(x_test_1_all_features_2D)

# MSE & MAE
mse = mean_squared_error(y_test_1, y_pred)
mae = mean_absolute_error(y_test_1, y_pred)

print(f"MSE: {round(mse, 4)}")
print(f"MAE: {round(mae, 4)}")

MSE: 18.8855
MAE: 3.1325


### Use Linear Regression to predict the next 6 hours

In [57]:
lin_reg = LinearRegression()

lin_reg.fit(x_train_6_all_features_2D, y_train_6)
lin_reg.score(x_train_6_all_features_2D, y_train_6)

0.4170238455461509

In [58]:
y_pred = lin_reg.predict(x_train_6_all_features_2D)

# MSE & MAE
mse = mean_squared_error(y_train_6, y_pred)
mae = mean_absolute_error(y_train_6, y_pred)

print(f"MSE: {round(mse, 4)}")
print(f"MAE: {round(mae, 4)}")

MSE: 24.1103
MAE: 3.6198


In [59]:
y_pred = lin_reg.predict(x_test_6_all_features_2D)

# MSE & MAE
mse = mean_squared_error(y_test_6, y_pred)
mae = mean_absolute_error(y_test_6, y_pred)

print(f"MSE: {round(mse, 4)}")
print(f"MAE: {round(mae, 4)}")

MSE: 42.0842
MAE: 4.3532


### Use XGBoost to predict the next 6 hours

In [60]:
xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

xgb_model.fit(x_train_6_all_features_2D, y_train_6)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, ...)

In [61]:
y_pred = xgb_model.predict(x_train_6_all_features_2D)

# MSE & MAE
mse = mean_squared_error(y_train_6, y_pred)
mae = mean_absolute_error(y_train_6, y_pred)

print(f"MSE: {round(mse, 4)}")
print(f"MAE: {round(mae, 4)}")

MSE: 0.0101
MAE: 0.071


In [62]:
y_pred = xgb_model.predict(x_test_6_all_features_2D)

# MSE & MAE
mse = mean_squared_error(y_test_6, y_pred)
mae = mean_absolute_error(y_test_6, y_pred)

print(f"MSE: {round(mse, 4)}")
print(f"MAE: {round(mae, 4)}")

MSE: 59.9362
MAE: 5.1039


In [63]:
# seems like XGBoost is easy to fall into the problem of "Overfitting." 
# a simple linear regression is more usefull in this case to predict the PM2.5 index 